In [ ]:
import pandas as pd

base = "https://raw.githubusercontent.com/jourdee-lab/FYP_Data_Pipeline/ecd59519e8f651e32ab2cafcfc4a0a7c16aa9eec/81sas04"


files = [
    f"{base}/81sas04ews_0.csv",
    f"{base}/81sas04ews_1.csv",
    f"{base}/81sas04ews_2.csv",
    f"{base}/81sas04ews_3.csv",
    f"{base}/81sas04ews_4.csv",
]

dfs = [
    pd.read_csv(
        url,
        engine="python",
        header=None,
        on_bad_lines="skip"
    )
    for url in files
]

sas04_1981 = pd.concat(dfs, ignore_index=True)


print(sas04_1981.shape)
print(sas04_1981.head())

manchester_codes = ["0C", "0D", "0E"]  # extend later if needed

far_east_1981 = sas04_1981.loc[
    sas04_1981[zone_col].isin(manchester_codes),
    [zone_col, "81sas040359", "81sas040360", "81sas040361"],
].copy()

far_east_1981.rename(columns={zone_col: "zoneid"}, inplace=True)
print(far_east_1981)



(141866, 61)
           0            1            2            3            4   \
0      zoneid  81sas040320  81sas040321  81sas040322  81sas040323   
1  0C             45771956     22288395     23483561     41084500   
2  0D              2749640      1336323      1413317       468000   
3  0E              5035315      2428472      2606843       297784   
4      zoneid  81sas040320  81sas040321  81sas040322  81sas040323   

            5            6            7            8            9   ...  \
0  81sas040324  81sas040325  81sas040326  81sas040327  81sas040328  ...   
1     19969127     21115373       731472       370516       360956  ...   
2       222860       245140        20716        10537        10179  ...   
3       144731       153053      4548708      2189833      2358875  ...   
4  81sas040324  81sas040325  81sas040326  81sas040327  81sas040328  ...   

            51           52           53           54           55  \
0  81sas040370  81sas040371  81sas040372  81sas0403

KeyError: 'zoneid'

In [3]:
import pandas as pd

base = "https://raw.githubusercontent.com/jourdee-lab/FYP_Data_Pipeline/ecd59519e8f651e32ab2cafcfc4a0a7c16aa9eec/81sas04"
files = [f"{base}/81sas04ews_{i}.csv" for i in range(5)]

dfs = [pd.read_csv(url, dtype={"zoneid": "string"}) for url in files]

sas04_1981 = pd.concat(dfs, ignore_index=True)

# clean column names just in case there is whitespace
sas04_1981.columns = sas04_1981.columns.str.strip()
sas04_1981["zoneid"] = sas04_1981["zoneid"].str.strip()

manchester_codes = ["0C", "0D", "0E"]

far_east_1981 = sas04_1981.loc[
    sas04_1981["zoneid"].isin(manchester_codes),
    ["zoneid", "81sas040359", "81sas040360", "81sas040361"],
].copy()

far_east_1981.rename(columns={
    "81sas040359": "far_east_total_1981",
    "81sas040360": "far_east_male_1981",
    "81sas040361": "far_east_female_1981",
}, inplace=True)

sas04_1981["zoneid"].astype(str).str.len().value_counts().sort_index()
sas04_1981["zoneid"].astype(str).head(20).tolist()


target_cols = ["81sas040359", "81sas040360", "81sas040361"]
missing = [c for c in target_cols if c not in sas04_1981.columns]
print("Missing:", missing)
print(sas04_1981[["zoneid"] + target_cols].head())



Missing: []
  zoneid  81sas040359  81sas040360  81sas040361
0     0C       122488        62925        59563
1     0D         4490         2507         1983
2     0E         9816         5220         4596
3     01        18627         9486         9141
4     02        19114         9590         9524


In [ ]:
import pandas as pd

base = "https://raw.githubusercontent.com/jourdee-lab/FYP_Data_Pipeline/ecd59519e8f651e32ab2cafcfc4a0a7c16aa9eec/81sas04"
files = [f"{base}/81sas04ews_{i}.csv" for i in range(5)]

dfs = [pd.read_csv(url, dtype={"zoneid": "string"}) for url in files]
sas04_1981 = pd.concat(dfs, ignore_index=True)

# Clean column names and zoneid
sas04_1981.columns = sas04_1981.columns.str.strip()
sas04_1981["zoneid"] = sas04_1981["zoneid"].str.strip()
value_cols = [c for c in sas04_1981.columns if c != "zoneid"]
sas04_1981[value_cols] = sas04_1981[value_cols].apply(pd.to_numeric, errors="coerce")

# Specify Manchester district
MANCHESTER_DISTRICT = "03BN"
manchester_zones = sas04_1981.loc[
    sas04_1981["zoneid"].str.startswith(MANCHESTER_DISTRICT),
    "zoneid"
].unique()

print(f"Found {len(manchester_zones)} Manchester zones")
print(f"Sample zones: {sorted(manchester_zones)[:5]}")

# Aggregate all Manchester zones
manchester_agg = pd.DataFrame(
    [sas04_1981.loc[sas04_1981["zoneid"].str.startswith(MANCHESTER_DISTRICT), value_cols].sum(numeric_only=True)],
    columns=value_cols
)

manchester_agg.insert(0, "zoneid", MANCHESTER_DISTRICT)

print("\n=== Manchester City (1981) - SAS04 Country of Birth ===")
print(manchester_agg)

# Extracting far east births only   
far_east_cols = ["81sas040359", "81sas040360", "81sas040361"]
far_east_agg = manchester_agg[["zoneid"] + far_east_cols].copy()
far_east_agg.columns = ["zoneid", "far_east_total", "far_east_male", "far_east_female"]

print("\n=== Far East births in Manchester (1981) ===")
print(far_east_agg)
manchester_agg.to_csv("manchester_sas04_1981.csv", index=False)
print("\nFull aggregate saved to: manchester_sas04_1981.csv")


Found 1053 Manchester zones
Sample zones: ['03BN', '03BNAA', '03BNAA01', '03BNAA02', '03BNAA03']

=== Manchester City (1981) - SAS04 Country of Birth ===
  zoneid  81sas040320  81sas040321  81sas040322  81sas040323  81sas040324  \
0   03BN      1312954       638492       674462      1118981       535563   

   81sas040325  81sas040326  81sas040327  81sas040328  ...  81sas040370  \
0       583418        22704        11775        10929  ...         6873   

   81sas040371  81sas040372  81sas040373  81sas040374  81sas040375  \
0         6545         2756         3789         7433         3904   

   81sas040376  81sas040377  81sas040378  81sas040379  
0         3529        15051         9737         5314  

[1 rows x 61 columns]

=== Far East births in Manchester (1981) ===
  zoneid  far_east_total  far_east_male  far_east_female
0   03BN            7142           4511             2631

Full aggregate saved to: manchester_sas04_1981.csv
